In [ ]:
# 바벨랭 발표자료 

### 파이썬 gensim을 사용해서 한국어 Word2Vec을 구현해보자.

우선 Word2Vec은 간단하게 말해서 단어들을 고정된 차원의 벡터스페이스에 유의미하게 배치해주는 word embedding 알고리즘 중 하나이다.

예를 들면 king + woman - queen = man이나 한국 - 서울 + 도쿄 = 일본이 성립하도록.




이를 구현하는 방법에는 주변 단어들로 빈 자리에 들어갈 단어를 맞추는 CBOW과 주어진 단어로 주변에 들어갈 단어를 맞추는 skip-gram이 있다.

이 글에선 Word2Vec 알고리즘을 내장하고 있는 gensim 라이브러리를 사용해서 빠르게 한국어 Word2Vec을 구현하는 것만 소개하도록 하겠다.

여기서 소개하는 구현법으로 그대로 하면 성능이 그닥 좋게 나오진 않는다.. 훌륭한 성능을 내고 있는 한국어 Word2Vec 사이트로는 elnn짱이 만든 Korean Word2Vec이 있으니 재밌는 결과가 보고싶다면 저쪽을 사용해보길!

gensim을 쓰기에 앞서 우선 corpus를 준비해야한다. 그리고 적당한 형태소 분석기를 사용해서 POS 태깅을 해주는게 좋다. 

같은 단어라도 품사가 다를 경우 다르게 인식해줘야하기 때문이다. 특히나 “제이미가"에서 "가"는 조사인데, 이런걸 나눠줄 필요가 있기때문에 더더욱 해야한다. 

영어였다면 단순한 tokenizing과 stemming으로도 충분 했을 것이다.

형태소 분석기별 차이는 여기에 자세하게 소개가 되어있으니 참고하면 좋다.


다만 Word2Vec의 주요 검색어는 명사가 될 것이기 때문에 (그 외 품사들은 학습용으로 사용된다고 생각하면 된다) 너무 세세하게 나눠주는 꼬꼬마나 한국어 Mecab보단 Twitter의 형태소 분석기가 더 적합하다고 판단했다.

In [32]:
#!pip install konlpy

In [33]:
#!pip install -U gensim

In [35]:
# -*- coding: utf-8 -*-
from konlpy.tag import Kkma
from konlpy.utils import pprint
import re

# corpus

In [2]:
from konlpy.corpus import kolaw

대한민국 헌법

In [3]:
doc = kolaw.open('constitution.txt').read()

지방공무원법 일부개정법률안

In [4]:
#from konlpy.corpus import kobill

In [5]:
#doc = kolaw.open('1809890.txt').read()

In [6]:
#print(doc)

# Tokenize

In [7]:
def text_cleaning(text):
    text = re.sub('[^가-힝0-9a-zA-Z]', ' ', text)
    return text

In [8]:
doc = text_cleaning(doc)

In [9]:
#print(doc)

문자열에 대해 명사 추출
KoNLPy를 통해 꼬꼬마 형태소 분석기를 이용하여 한글 문자열로부터 명사를 얻을 수 있다.

In [10]:
#nouns = Kkma().nouns(doc)

In [11]:
nouns = Kkma().pos(doc)

In [12]:
print(nouns)

[('대한민국', 'NNG'), ('헌법', 'NNG'), ('유구', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('역사', 'NNG'), ('와', 'JC'), ('전통', 'NNG'), ('에', 'JKM'), ('빛나', 'VV'), ('는', 'ETD'), ('우리', 'NNM'), ('대하', 'VV'), ('ㄴ', 'ETD'), ('국민', 'NNG'), ('은', 'JX'), ('3', 'NR'), ('1', 'NR'), ('운동', 'NNG'), ('으로', 'JKM'), ('건립', 'NNG'), ('되', 'XSV'), ('ㄴ', 'ETD'), ('대한민국', 'NNG'), ('임시', 'NNG'), ('정부', 'NNG'), ('의', 'JKG'), ('법통', 'NNG'), ('과', 'JC'), ('불의', 'NNG'), ('에', 'JKM'), ('항거', 'NNG'), ('하', 'XSV'), ('ㄴ', 'ETD'), ('4', 'NR'), ('19', 'NR'), ('민주', 'NNG'), ('이념', 'NNG'), ('을', 'JKO'), ('계승', 'NNG'), ('하', 'XSV'), ('고', 'ECE'), ('조국', 'NNG'), ('의', 'JKG'), ('민주', 'NNG'), ('개혁', 'NNG'), ('과', 'JKM'), ('평화적', 'NNG'), ('통일', 'NNG'), ('의', 'JKG'), ('사명', 'NNG'), ('에', 'JKM'), ('입각', 'NNG'), ('하', 'XSV'), ('여', 'ECS'), ('정의', 'NNG'), ('인도', 'NNG'), ('와', 'JC'), ('동포애', 'NNG'), ('로써', 'JKM'), ('민족', 'NNG'), ('의', 'JKG'), ('단결', 'NNG'), ('을', 'JKO'), ('공고히', 'MAG'), ('하', 'VV'), ('고', 'ECE'), ('모든', 'MDT'), ('사회적', 'NNG')

In [13]:
type(nouns)

list

In [14]:
len(nouns)

9200

2013년 구글에서 개발한 Word2Vec이라는 방법론이 있습니다. 이름 그대로 단어(Word)를 벡터(Vector)로 바꿔주는 방법입니다. 이를 임베딩(Embedding)이라고 합니다

word2vec은 한 문장 내에서 단어의 등장 빈도 데이터를 바탕으로 이를 단어를 벡터 스페이스에 embedding 하는 작업입니다.

neural network를 통해 각 단어가 같이 등장한 빈도가 높으면 두 단어 벡터의 거리를 가깝게 하고, 빈도가 낮으면 두 닺어 벡터의 거리를 멀게함을 통해 단어벡터를 만드는 것입니다

주지하다시피 Word2Vec의 효과는 놀랍습니다. 

단어를 벡터화할 때 단어의 문맥적 의미를 보존하기 때문이죠. 위의 그림처럼 MAN과 WOMAN 사이의 거리는 KING과 QUEEN 거리와 유사합니다. 벡터로 바뀐 단어들은 ‘유클리디안 거리’니, ‘코사인 유사도’니 하는 방식들로 그 거리를 잴 수 있고 그 거리가 가까울(작을) 경우 의미가 비슷한 단어라고 해석할 수 있게 됩니다.

컨텐츠간 유사도는 컨텐츠 벡터간 cosine similarity로 정의했습니다. 두 벡터의 내적을 통해 두 벡터간 코사인 값을 계산하고이를 유사도라고 정의했습니다. 즉 동일한 컨텐츠에 대해서는 1, 정 반대의 컨텐츠에 대해서는 -1 유사도를 갖습니다.

* 코사인 유사도

코사인 유사도(― 類似度, 영어: cosine similarity)는 내적공간의 두 벡터간 각도의 코사인값을 이용하여 측정된 벡터간의 유사한 정도를 의미한다. 각도가 0°일 때의 코사인값은 1이며, 다른 모든 각도의 코사인값은 1보다 작다. 따라서 이 값은 벡터의 크기가 아닌 방향의 유사도를 판단하는 목적으로 사용되며, 두 벡터의 방향이 완전히 같을 경우 1, 90°의 각을 이룰 경우 0, 180°로 완전히 반대 방향인 경우 -1의 값을 갖는다.

![agTBWiT.png](attachment:agTBWiT.png)

# Training

Word Embedding : 문장속의 단어간의 관계를 비지도 학습 방식으로 분석하여 만들어지는 수십~ 수백차원의 벡터로서 특징(Feature)화 되는 단어들을 만들어 냅니다. 단어가 가지고있는 벡터간의 연산을 통해 다른 단어와의 관계를 만들어내게 됩니다.
* 하단 설명 참조

Topic Modeling : 어떤 문서에서 자주 나타는 단어를 통해, 주제를 찾아주는 확률적인 모델을 디자인합니다. 유명한 것으로 LDA, LSI, HDP가 있습니다.

In [15]:
import gensim

In [25]:
# Word2Vec embedding
from gensim.models import Word2Vec
model = Word2Vec(nouns, size=100, window = 3, min_count=10, workers=4, iter=100, sg=1)

위 코드 의미는 다음과 같습니다. 포스태깅된 컨텐츠를 100차원의 벡터로 바꿔라. 주변 단어(window)는 앞뒤로 두개까지 보되, 코퍼스 내 출현 빈도가 50번 미만인 단어는 분석에서 제외해라. CPU는 쿼드코어를 쓰고 100번 반복 학습해라. 분석방법론은 CBOW와 Skip-Gram 중 후자를 선택해라.

In [26]:
model.wv.vocab

{'1': <gensim.models.keyedvectors.Vocab at 0x12a38f5f8>,
 '2': <gensim.models.keyedvectors.Vocab at 0x12a3920b8>,
 '3': <gensim.models.keyedvectors.Vocab at 0x12a38f588>,
 '4': <gensim.models.keyedvectors.Vocab at 0x12a38f780>,
 '5': <gensim.models.keyedvectors.Vocab at 0x12a392358>,
 '6': <gensim.models.keyedvectors.Vocab at 0x12a392400>,
 'ECD': <gensim.models.keyedvectors.Vocab at 0x12a38fac8>,
 'ECE': <gensim.models.keyedvectors.Vocab at 0x12a38f860>,
 'ECS': <gensim.models.keyedvectors.Vocab at 0x12a38f8d0>,
 'ETD': <gensim.models.keyedvectors.Vocab at 0x12a38f2e8>,
 'ETN': <gensim.models.keyedvectors.Vocab at 0x12a38fc18>,
 'JC': <gensim.models.keyedvectors.Vocab at 0x12a38f358>,
 'JKG': <gensim.models.keyedvectors.Vocab at 0x12a38f710>,
 'JKM': <gensim.models.keyedvectors.Vocab at 0x12a38f3c8>,
 'JKO': <gensim.models.keyedvectors.Vocab at 0x12a38f7f0>,
 'JKS': <gensim.models.keyedvectors.Vocab at 0x12a3921d0>,
 'JX': <gensim.models.keyedvectors.Vocab at 0x12a38f550>,
 'MAG': <ge

In [27]:
#model.save('model')

In [28]:
#new_model = gensim.models.Word2Vec.load('model')

In [29]:
print(model)

Word2Vec(vocab=161, size=100, alpha=0.025)


그럼 임베딩이 잘 됐는지 확인해볼까요? gensim 패키지가 제공하는 기능 중에 ‘most_similar’라는 함수가 있습니다. 두 벡터 사이의 코사인 유사도를 구해줍니다. 그 값이 작을 수록 비슷한 단어라는 뜻인데, 아래 코드는 ‘디자인’이라는 단어와 가장 비슷한(코사인 유사도가 큰) 100개 단어를 출력하라는 지시입니다.

In [30]:
# check embedding result

try:
        print(model.wv.most_similar(positive=["국민"], topn=100))
except KeyError:
        print("not in vocabulary")

[('의원', 0.9937130212783813), ('의무', 0.9923284649848938), ('공포', 0.9922620058059692), ('안', 0.9915605783462524), ('행정', 0.9914844632148743), ('국무', 0.9914082288742065), ('공무원', 0.991243839263916), ('국회', 0.9904596209526062), ('재판', 0.9901857376098633), ('관리', 0.9901809096336365), ('직무', 0.9900108575820923), ('범위', 0.9899652004241943), ('회의', 0.9897454380989075), ('국회의원', 0.9893331527709961), ('법원', 0.9890962243080139), ('기관', 0.9887917637825012), ('찬성', 0.9886487126350403), ('탄핵', 0.9884301424026489), ('제', 0.9882793426513672), ('항', 0.9880039095878601), ('보장', 0.9878906011581421), ('대통령', 0.9878782033920288), ('국무총리', 0.9878024458885193), ('법률', 0.9876260757446289), ('헌법', 0.9875929355621338), ('국', 0.987295925617218), ('노력', 0.9872297644615173), ('위원', 0.9871621131896973), ('임명', 0.987145721912384), ('조약', 0.9869192838668823), ('보호', 0.98687344789505), ('조직', 0.9868249893188477), ('의결', 0.9868209958076477), ('기타', 0.9864730834960938), ('국가', 0.9863284826278687), ('단체', 0.9862365126609

여튼 이제 이 model이라고 써둔 Word2Vec 객체만 있으면 여러가지를 할 수 있다. 가령 맨 위에 소개해둔 한국 - 서울 + 도쿄를 계산하려면

In [31]:
model.wv.most_similar(positive=["국민", "대통령"], negative=["헌법"], topn=1)

[('의원', 0.9868676662445068)]

word2vec의 흥미로운 점은 단어 벡터들끼리의 연산이 의미있는 결과를 가져온다는 점입니다. word2vec으로 학습시킨 벡터들을 연산하면 다음과 같은 결과가 나옵니다.

vector('Paris') - vector('France') + vector('Italy') = vector('Rome')

vector('king') - vector('man') + vector('woman') = vector('queen')

이처럼 word2vec 은 단순한 단어의 벡터 표현이 아니라 단어의 추론, 복잡한 개념까지도 표현이 가능합니다.

### word embedding

단어 임베딩(Word Embedding)이란 텍스트를 구성하는 하나의 단어를 수치화하는 방법의 일종이다.

텍스트 분석에서 흔히 사용하는 방식은 단어 하나에 인덱스 정수를 할당하는 Bag of Words 방법이다. 이 방법을 사용하면 문서는 단어장에 있는 단어의 갯수와 같은 크기의 벡터가 되고 단어장의 각 단어가 그 문서에 나온 횟수만큼 벡터의 인덱스 위치의 숫자를 증가시킨다.

즉 단어장이 "I", "am", "a", "boy", "girl" 다섯개의 단어로 이루어진 경우 각 단어에 다음과 같이 숫자를 할당한다.

"I": 0
"am": 1
"a": 2
"boy": 3 
"girl": 4

이 때 "I am a girl" 이라는 문서는 다음과 같이 벡터로 만들 수 있다.

[11101]

 
단어 임베딩은 하나의 단어를 하나의 인덱스 정수가 아니라 실수 벡터로 나타낸다. 예를 들어 2차원 임베딩을 하는 경우 다음과 같은 숫자 벡터가 될 수 있다.

"I": (0.3, 0.2)
"am": (0.1, 0.8)
"a": (0.5, 0.6)
"boy": (0.2, 0.9) 
"girl": (0.4, 0.7)

단어 임베딩이 된 경우에는 각 단어 벡터를 합치거나(concatenation) 더하는(averaging, normalized Bag of Words) 방식으로 전체 문서의 벡터 표현을 구한다

### word2vec

word2vec은 CBOW 방식과 Skip-Gram 방식의 단어 임베딩을 구현한 C++ 라이브러리로 구글에 있던 Mikolov 등이 개발하였다.

파이썬에서는 gensim이라는 패키지에 Word2Vec이라는 클래스로 구현되어 있다

word2vec 모델의 구조는 CBOW(Continuous Bag of Words), Continuous Skip-Gram 2 가지 방식이 있습니다. 

아래의 예시를 통해 2가지 방식의 차이점을 쉽게 알 수 있습니다.

* 문장 : When I find myself in times of trouble mother Mary comes to me

CBOW Input : [When, I, find], [I, find, myself], [myself, in, times], ...

Skip-Gram Input : [When, I, find], [When, I, myself], [When, find, myself], ...

위의 예시 처럼 CBOW 방식은 한 문장에서 연속적인 단어들을 입력값으로 집어 넣는 것으로 문맥을 통해 단어를 예측할 수 있는 모델입니다. 

Skip-Gram은 이와 반대로 한 문장 안에서 발생할 수 있는 단어 모음(Bag of words)를 모두 입력값으로 취하기에 단어를 통해 문맥을 예측할 수 있는 모델입니다. 

CBOW는 입력값이 Skip-Gram보다 적기 때문에 학습 시간이 더 짧고 잦은 빈도로 등장하는 단어에 대해 정확한 결과를 도출해낼 수 있습니다. 

Skip-Gram은 학습 시간이 더 길지만 작은 데이터셋에 대해 효율적이고 자주 등장하지 않는 단어에 대해 더 높은 정확도를 갖습니다.




### CBOW (Continuous Bag of Words) Embedding

위의 방식은 하나의 단어로부터 다음에 오는 단어를 예측하는 문제였다. 이러한 문제를 단어 하나짜리 문맥(single-word context)를 가진다고 한다.

CBOW (Continuous Bag of Words) 방식은 복수 단어 문맥(multi-word context)에 대한 문제 즉, 여러개의 단어를 나열한 뒤 이와 관련된 단어를 추정하는 문제이다. 즉, 문자에서 나오는  nn 개의 단어 열로부터 다음 단어를 예측하는 문제가 된다. 예를 들어

* the quick brown fox jumped over the lazy dog

라는 문장에서 (the, quick, brown) 이라는 문맥이 주어지면 fox라는 단어를 예측해야 한다.

CBOW는 다음과 같은 신경망 구조를 가진다. 여기에서 각 문맥 단어를 은닉층으로 투사하는 가중치 행렬은 모든 단어에 대해 공통으로 사용한다.


### Skip-Gram Embedding

Skip-Gram 방식은 CBOW 방식과 반대로 특정한 단어로부터 문맥이 될 수 있는 단어를 예측한다. 보통 입력 단어 주변의  kk 개 단어를 문맥으로 보고 예측 모형을 만드는데 이  kk  값을 window size 라고 한다.

위 문장에서 window size  k=1인 경우,

quick -> the
quick -> brown
brown -> quick
brown -> fox

과 같은 관계를 예측할 수 있어야 한다.


text classification을 하고 있습니다. 
text 내용을 분석하여 category label을 예측하는 내용인데, 이런 경우 대걔 어떤 방법으로 접근하는 것이 좋은 결과를 볼 수 있는지 고수님들의 경험담을 듣고 싶습니다.

제가 해본 것은 
1. Tensorflow CNN word2vec 기반
2. Scikit-learn SVM (linear-kernel) 기반
3. Scikit-learn SVM (rbf-kernel) 기반
4. Scikit-learn MultinomialNB TF-IDF 기반
정도인데 (파라미터 튜닝은 각각 어느 정도 해본 상태에서) 4번 결과가 제일 좋네요.